<a href="https://colab.research.google.com/github/Jaejuna/SpeedWagon/blob/main/SentimentClassification/Sentiment_Classification(leun).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/SKTBrain/KoBERT/issues/67  
https://complexoftaste.tistory.com/2

# KOBERT 모델로 감성분류(7가지 감성)

### 환경설정

In [1]:
# !pip install mxnet
# !pip install gluonnlp==0.8.0
# !pip install tqdm pandas
# !pip install sentencepiece
# !pip install transformers
# !pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached gluonnlp-0.8.0-py3-none-any.whl
  Attempting uninstall: gluonnlp
    Found existing installation: gluonnlp 0.9.1
    Uninstalling gluonnlp-0.9.1:
      Successfully uninstalled gluonnlp-0.9.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install transformers==4.25.1
!pip install gluonnlp==0.9.1
!pip install --no-cache-dir transformers sentencepiece
!pip install mxnet
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#!git clone https://github.com/SKTBrain/KoBERT

In [9]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-qhv_qkmm/kobert-tokenizer_513098cdb677442783fa64d96f3f2be1
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-qhv_qkmm/kobert-tokenizer_513098cdb677442783fa64d96f3f2be1
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=3e578498a01601cba94497838248387504d657970684f24edda4710b6862e272
  Stored in directory: /tmp/pip-ephem-wheel-cache-yl46tust/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [2]:
import transformers
transformers.__version__

'4.25.1'

In [10]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel

### hugingface에서 model, tokenizer 불러오기

In [11]:
from transformers import AutoTokenizer, AutoModel
model_checkpoints = "skt/kobert-base-v1"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoints,use_fast=False)
model = AutoModel.from_pretrained(model_checkpoints)

In [6]:
def get_kobert_model(model_path, vocab_file, ctx="cpu"):
    bertmodel = BertModel.from_pretrained(model_path)
    device = torch.device(ctx)
    bertmodel.to(device)
    bertmodel.eval()
    vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(vocab_file,
                                                         padding_token='[PAD]')
    return bertmodel, vocab_b_obj

In [12]:
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel, vocab = get_kobert_model('skt/kobert-base-v1',tokenizer.vocab_file)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 데이터 입력

In [14]:
data = pd.read_csv('/content/drive/MyDrive/BOAZ_miniProject2/5차년도_2차.csv',encoding='cp949')

In [15]:
data.loc[(data['상황'] == "fear"), '상황'] = 0  #공포 => 0
data.loc[(data['상황'] == "surprise"), '상황'] = 1  #놀람 => 1
data.loc[(data['상황'] == "angry"), '상황'] = 2  #분노 => 2
data.loc[(data['상황'] == "sadness"), '상황'] = 3  #슬픔 => 3
data.loc[(data['상황'] == "neutral"), '상황'] = 4  #중립 => 4
data.loc[(data['상황'] == "happiness"), '상황'] = 5  #행복 => 5
data.loc[(data['상황'] == "disgust"), '상황'] = 6  #혐오 => 6

In [16]:
data_list = []
for ques, label in zip(data['발화문'], data['상황'])  :
    data = []   
    data.append(ques)
    data.append(str(label))

    data_list.append(data)

In [17]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         

    def __len__(self):
        return (len(self.labels))

In [18]:
from transformers import AutoModel, AutoTokenizer

kobert_tokenizer = AutoTokenizer.from_pretrained("skt/kobert-base-v1", use_fast=False)
kobert = AutoModel.from_pretrained("skt/kobert-base-v1")

result = kobert_tokenizer.tokenize("너는 내년 대선 때 투표할 수 있어?")
print(result)
kobert_vocab = kobert_tokenizer.get_vocab()
print(kobert_vocab.get('▁대선'))
print([kobert_tokenizer.encode(token) for token in result])

['▁너', '는', '▁내년', '▁대선', '▁때', '▁투표', '할', '▁수', '▁있어', '?']
1654
[[1457, 3, 2], [517, 5760, 3, 2], [1437, 3, 2], [1654, 3, 2], [1844, 3, 2], [4772, 3, 2], [4977, 3, 2], [2872, 3, 2], [3868, 3, 2], [633, 3, 2]]


In [19]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [20]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=34)

In [21]:
tok=tokenizer.tokenize
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test,0, 1, tok, vocab,  max_len, True, False)

In [22]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [23]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [26]:
#BERT 모델 불러오기
device = torch.device("cuda:0")
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


### 모델 학습

In [27]:
train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
         
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

<ipython-input-27-225f425166d4>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/243 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.970080018043518 train acc 0.140625


KeyboardInterrupt: ignored

### 예측

In [ ]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        
        valid_length = valid_length
        label = label.long().to(device)
        
        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("공포가")
            elif np.argmax(logits) == 1:
                test_eval.append("놀람이")
            elif np.argmax(logits) == 2:
                test_eval.append("분노가")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 4:
                test_eval.append("중립이")
            elif np.argmax(logits) == 5:
                test_eval.append("행복이")
            elif np.argmax(logits) == 6:
                test_eval.append("혐오가")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

In [ ]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        
        valid_length = valid_length
        label = label.long().to(device)
        
        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0: # 공포
                test_eval.append(0)
            elif np.argmax(logits) == 1: # 놀람
                test_eval.append(1)
            elif np.argmax(logits) == 2: # 분노
                test_eval.append(2)
            elif np.argmax(logits) == 3: # 슬픔
                test_eval.append(3)
            elif np.argmax(logits) == 4: # 중립
                test_eval.append(4)
            elif np.argmax(logits) == 5: # 행복
                test_eval.append(5)
            elif np.argmax(logits) == 6: # 혐오
                test_eval.append(6)

    return test_eval[0]

In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 그때 츌장갓다온 날 회사 안 가고 째고 #@이름#랑 롯데몰에서 만낫을때 아 멘무샤인가 거기 갔지 응 그거 없어짐 헐 슬프군 롯데몰에 멕시칸음식점이랑 다양하게 생겼던데 담에 도전해보세 오 멕시칸 좋아 온더보더 생겻나 응 어 그거 생긴듯 온더보더 존좋
>> 입력하신 내용에서 분노가 느껴집니다.


하고싶은 말을 입력해주세요 : 충전기 차량용은 잘안파네 충전못해 아니해 내차꺼 망가져서 기왕사는거 차량용사려했는데 코드로된거밖에안파넹 인터넷에서 사야겠네 충전시쟉 놀면서가겟네
>> 입력하신 내용에서 분노가 느껴집니다.


하고싶은 말을 입력해주세요 : 이미 엄니랑 누나한테 내가요리해준다고 말해놨음 미리 해보구 감동적인거 아니냐구 안해봐도 되는디 헐 누나 표정이 니가 엄니는 좋아하심 진짜 웃겨 남자도 요리할줄 알아야한다고
>> 입력하신 내용에서 분노가 느껴집니다.


하고싶은 말을 입력해주세요 : 헐 행복해
>> 입력하신 내용에서 행복이 느껴집니다.


하고싶은 말을 입력해주세요 : 0


**문장 예시**
- 그때 츌장갓다온 날 회사 안 가고 째고 #@이름#랑 롯데몰에서 만낫을때 아 멘무샤인가 거기 갔지 응 그거 없어짐 헐 슬프군 롯데몰에 멕시칸음식점이랑 다양하게 생겼던데 담에 도전해보세 오 멕시칸 좋아 온더보더 생겻나 응 어 그거 생긴듯 온더보더 존좋
- 충전기 차량용은 잘안파네 충전못해 아니해 내차꺼 망가져서 기왕사는거 차량용사려했는데 코드로된거밖에안파넹 인터넷에서 사야겠네 충전시쟉 놀면서가겟네
- 이미 엄니랑 누나한테 내가요리해준다고 말해놨음 미리 해보구 감동적인거 아니냐구 안해봐도 되는디 헐 누나 표정이 니가 엄니는 좋아하심 진짜 웃겨 남자도 요리할줄 알아야한다고

In [ ]:
sum_train = pd.read_csv('/content/drive/MyDrive/BOAZ_miniProject2/train.csv')
sum_valid = pd.read_csv('/content/drive/MyDrive/BOAZ_miniProject2/valid.csv')

In [ ]:
sum_train['Text']

0         웃긴게 20년동안... 바로 옆 가게 주차장에서 같이 그 감자탕집 주차도 했었는데 ...
1         아까 보니깐 로켓 프레시 잇드라 #@시스템#사진# 나도 밥 무거요 호앵 야 제주도는...
2         #@이름#아 먼가 나머리망 이불위에 두고 온거 같다 확인좀 ㅠ 읭?? 만약에 잇으면...
3         분양가 얼마여써? 여기 일억육천구백이더라ㅔ ㅋㅋㅋㅋㅋ걍 여기서 살고싶다..........
4         지금 뭔가 카톡 느리지않녀 새해라 다들 톡하느라 터진건가 ㅇㅇ 약간 버벅댐 ㅇㅇㅇ ...
                                ...                        
279987    내가 좋아하는 댄서 있는데 그 사람은 춤을 어떻게 그렇게 잘 추는 몰라 열심히 연습...
279988    붐 진짜 마돈나돈나~돈나~막이래 모든노래에!! 지금은 소주한잔 부르고 있어.. 대단...
279989    빈둥거리고 놀다가 영화보구 나왔어 완벽한 타인 봤는데 겁나 쓰레긴데 개웃겨 ㅋㅋㅋㅋ...
279990    머보고싶냥~~~ 와우 너가 보고싶은거 봐야하는거 아녀?.. ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 스...
279991    아 끝났다 블로그 쓰기 한시간 걸렸다 앜ㅋㅋㅋ 원래그정도야??? ㅋㅋㅋㅋㅋㅋ아닠ㅋㅋ...
Name: Text, Length: 279992, dtype: object

### 요약 데이터 전처리
https://velog.io/@ryan_95/%EA%B0%90%EC%84%B1-%EB%B6%84%EC%84%9D-%EB%AA%A8%EB%8D%B8-%EC%84%B1%EB%8A%A5%ED%96%A5%EC%83%81%EC%9D%84-%EC%9C%84%ED%95%9C-%EC%97%B0%EA%B5%AC

https://www.itbiznews.com/news/articleView.html?idxno=65720  

https://wikidocs.net/44249  

https://github.com/kimwoonggon/publicservant_AI/blob/master/2_(Hugging%2B%ED%95%9C%EA%B8%80BERT)%EB%84%A4%EC%9D%B4%EB%B2%84%20%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D%2090%25%20%EB%8B%AC%EC%84%B1%ED%95%98%EA%B8%B0.ipynb


https://velog.io/@fhflwhwl5/Python-KoBERT-7%EA%B0%80%EC%A7%80-%EA%B0%90%EC%A0%95%EC%9D%98-%EB%8B%A4%EC%A4%91%EA%B0%90%EC%84%B1%EB%B6%84%EB%A5%98%EB%AA%A8%EB%8D%B8-%EA%B5%AC%ED%98%84%ED%95%98%EA%B8%B0

---
CNN  
https://wonhwa.tistory.com/35  
konlpy  
https://cyc1am3n.github.io/2018/11/10/classifying_korean_movie_review.html

In [ ]:
def preprocess_sentence(sentence):
    sentence = sentence.lower() # 텍스트 소문자화
    sentence = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]+[/ㄱ-ㅎㅏ-ㅣ]', '', sentence) # 여러개 자음과 모음을 삭제한다.
    sentence = re.sub("[^가-힣a-z0-9#@,-]", " ", sentence) # 영어 외 문자(숫자, 특수문자 등) 공백으로 변환
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개 공백을 하나의 공백으로 바꿉니다.
    sentence = sentence.strip() # 문장 양쪽 공백 제거
    
    return sentence

def data_process(data):
  # 전체 Text 데이터에 대한 전처리 (1)
  text = []

  for data_text in tqdm(data):
    text.append(preprocess_sentence(data_text))
  
  return text

train_texts = data_process(sum_train['Text'])
val_texts = data_process(sum_valid['Text'])

train_df = pd.DataFrame(zip(train_texts,train['Summary']), columns=['Text', 'Summary'])
val_df = pd.DataFrame(zip(val_texts,valid['Summary']), columns=['Text', 'Summary'])

### (todo)확률 추가하는게 좋지 않을까?하는 생각

In [ ]:
sum_valid['Text']

In [ ]:
valid_res = []
for txt in sum_valid['Text']:
  valid_res.append(predict(txt))

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


KeyboardInterrupt: ignored

In [ ]:
len(valid_res) # 약 1시간 20분정도 돌리고 3710개 완료(마지막은 혹시 모르니 제외)

3711

In [ ]:
sum_valid_new = sum_valid[0:3710]

In [ ]:
sum_valid_new = sum_valid_new.copy()
sum_valid_new['sentiment'] = valid_res[0:3710]

In [ ]:
sum_valid_new.head()

,Text,Summary,Topic,sentiment
0,수유로오셈^^ 지금? 지금가도대? 지금이겠냐? 되겠냐? 집 데려다주냐? 왜안댐? ㅋ...,지금 수유에 갈 테니 집에 데려다 달라고 한다.,행사,2
1,제주도가구시퍼져써 여보시간 업자나 2박3일정두 갔다올수이ㅛ을거같운데 쉬는날몰아서 구...,2박 3일 정도 쉬는 날을 몰아서 제주도에 갔다 오기로 했다.,행사,2
2,무튼 몇시에 어디서 만나지? 정한거 아냐? 메가박스 2시 45분 매표소?역출구? 매...,메가박스 매표소에서 2시 45분에 만나기로 약속했다.,행사,4
3,낼상황봐서 정하자 오켕 낼 보구 밥 정하쟈 ~ 웅 조아용 그리구 간지카페가자 힐링해...,내일 상황을 봐서 정하자고 카페를 가서 힐링해야 한다고 한다.,행사,2
4,내일 모레 머먹어? 김장김치랑 수육!!!!!!!!!! 수!!! 육!!!!! 나는? ...,모레 김장 김치와 수육을 먹을 테니 집으로 오라고 한다.,행사,2


In [ ]:
sum_valid_new.to_csv('/content/drive/MyDrive/BOAZ_miniProject2/valid_res1.csv')